# BERLIN // Where to open a new restaurant: neighborhood analysis

# 1 - Introduction

## 1.1 Discussion of the problem

After COVID pandemic, more problematic waves have passed and once the vaccination process is fully advanced, a well known spanish restaurant chain company is planning to open a new restaurant in Berlin. What will be the best place to do it?

## 1.2 Discussion of the background

During 2020 tourism was one the main industries affected by lockdown decisions around the world. According to UN data, during 2020 international arrivals are estimated to have dropped to 381 million, down from 1.461 billion $ in 2019 — a 74% decline. In countries whose economies are heavily reliant on tourism as the suth of Europe (Italy, Portugal, Greece or Spain), the precipitous drop in visitors was, and remains, devastating.

Berlin was not left out of this huge crisis. It is the capital and the biggest city of Germany, the second most populous city in the European Union, Berlin has nearly 3,6 million residents from more than 190 countries with a population density of 4,200 people per km², the city is divided into 12 boroughs, 95 neighborhoods. 
Also it is considered a top European destination – ranked third after London and Paris.

During 2020 even though the world is facing the Coronavirus crisis, Berlin welcomed almost 5 millions tourists in the whole year 2020, which represents a decrease of 65% of the same period in 2019.
At the beggining of 2021, between January and April 400,000 tourists have visited Berlin, and it is expected these figures could rise as vaccination process improves and frontiers are widely opened.
There are more than 1.300 restaurants in Berlin.

In order to face this issue, we can solve this problem by creating a map and information chart that shows the real distribution of these restaurants in Berlin and clustering each area according to the density of the place.
We will need to find a method to use Foursquare location data where machine learning to help us make decisions for the spanish restaurant company. 

In this project, I will try to use Foursquare location data and clustering methods to divide regions into different groups based on their restaurant location information.

# 2 - Data description: how it helps to solve the problem

For this project, data needed is as follows:

**1 - Berlin neigborood data: list of Boroughs and neighborhoods and their latitudes and longitudes.**
<ul>
<li> Data source: https://en.wikipedia.org/wiki/Boroughs_and_neighborhoods_of_Berlin </li>
<li> Description: We will discard the Berlin area (district) table through Wikipedia. Then using geocoder class of the Geopy to get coordinates (lattitude and longited) of these 12 main areas. </li>
</ul>
    
**2 - Restaurants in each neighborhood in Berlin:**

<ul>
<li> Data source: Foursquare API </li>
<li> Description: By using this API, we will obtain all venues in each community. We can filter these places to get only restaurants. </li>
</ul>

# 3 - Methodology

## 3.1 Getting information from Berlin's neighborhood

First of all, we get information about boroughs and neighborhood of Berlin scrapped from Wikipedia

In [1]:
!pip install bs4
from bs4 import BeautifulSoup
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

In [2]:
!wget -O berlin.html https://en.wikipedia.org/wiki/Boroughs_and_neighborhoods_of_Berlin

--2021-07-26 15:00:04--  https://en.wikipedia.org/wiki/Boroughs_and_neighborhoods_of_Berlin
Resolving en.wikipedia.org (en.wikipedia.org)... 208.80.154.224, 2620:0:861:ed1a::1
Connecting to en.wikipedia.org (en.wikipedia.org)|208.80.154.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 207266 (202K) [text/html]
Saving to: ‘berlin.html’

berlin.html         100%[===================>] 202.41K   627KB/s    in 0.3s    

2021-07-26 15:00:05 (627 KB/s) - ‘berlin.html’ saved [207266/207266]



Parse the html file

In [3]:
with open('berlin.html','r') as berlin_html:
    soup_berlin = BeautifulSoup(berlin_html, 'html.parser')

Create a dataframe with the list of neighbourhoods from the html file

In [4]:
df_berlin = []
for tr in soup_berlin.find_all('tr'):
    row = tr.text.replace('(','').replace(')','')
    row = row.split('\n')
    row = list(filter(lambda s: s != '', row)) # delete empty strings from list
    row = list(map(lambda s: s.strip(), row)) # remove leading and trailing spaces from strings in list
                 
    if row[0][0:4].isdigit():
        row = row[0].split(' ', 1)
        df_berlin.append(row)

df_berlin = pd.DataFrame(df_berlin)
df_berlin.columns = ['neighborhood_id', 'neighborhood']

Get list of boroughs in ID order and add to each neighbourhood

In [5]:
boroughs = []
for dt in soup_berlin.find_all('dt'):
    boroughs.append(dt.text[5:])

# add borough 
borough = []
for lid in df_berlin.neighborhood_id:
    borough.append(boroughs[int(lid)//100-1])
    
df_berlin['borough'] = borough
df_berlin['city'] = 'Berlin'

df_berlin

,neighborhood_id,neighborhood,borough,city
0,0101,Mitte,Mitte,Berlin
1,0102,Moabit,Mitte,Berlin
2,0103,Hansaviertel,Mitte,Berlin
3,0104,Tiergarten,Mitte,Berlin
4,0105,Wedding,Mitte,Berlin
...,...,...,...,...
91,1207,Waidmannslust,Reinickendorf,Berlin
92,1208,Lübars,Reinickendorf,Berlin
93,1209,Wittenau,Reinickendorf,Berlin
94,1210,Märkisches Viertel,Reinickendorf,Berlin


## 3.2 Adding coordinates for each neighborhood

Now the aim is to add coordinates details for each of 96 neighborhoods. We will use Geopy client detailes as follows. In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent <em>be_explorer</em>, as shown below.

In [6]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

!conda install -c conda-forge folium=0.5.0 --yes 
import folium

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



Getting information about Berlin coordinates:

In [7]:
address = 'Berlin, Germany'

geolocator = Nominatim(user_agent="be_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Berlin are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Berlin are 52.5170365, 13.3888599.


Now details about latitude and longitude for all neighborhoods

In [8]:
geolocator = Nominatim(user_agent="be_explorer")

df_berlin['neighborhood_coord']= df_berlin['neighborhood'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))
df_berlin[['Latitude', 'Longitude']] = df_berlin['neighborhood_coord'].apply(pd.Series)

df_berlin

,neighborhood_id,neighborhood,borough,city,neighborhood_coord,Latitude,Longitude
0,0101,Mitte,Mitte,Berlin,"(39.98020495, -7.905590887431517)",39.980205,-7.905591
1,0102,Moabit,Mitte,Berlin,"(52.5301017, 13.3425422)",52.530102,13.342542
2,0103,Hansaviertel,Mitte,Berlin,"(52.5191234, 13.3418725)",52.519123,13.341872
3,0104,Tiergarten,Mitte,Berlin,"(50.3409222, 6.956329)",50.340922,6.956329
4,0105,Wedding,Mitte,Berlin,"(52.550123, 13.34197)",52.550123,13.341970
...,...,...,...,...,...,...,...
91,1207,Waidmannslust,Reinickendorf,Berlin,"(52.6080354, 13.3225327)",52.608035,13.322533
92,1208,Lübars,Reinickendorf,Berlin,"(52.6146467, 13.3530197)",52.614647,13.353020
93,1209,Wittenau,Reinickendorf,Berlin,"(52.5912366, 13.3233195)",52.591237,13.323320
94,1210,Märkisches Viertel,Reinickendorf,Berlin,"(52.5993123, 13.3565324)",52.599312,13.356532


In [9]:
df_berlin.drop(['neighborhood_coord'], axis=1, inplace=True)
df_berlin

,neighborhood_id,neighborhood,borough,city,Latitude,Longitude
0,0101,Mitte,Mitte,Berlin,39.980205,-7.905591
1,0102,Moabit,Mitte,Berlin,52.530102,13.342542
2,0103,Hansaviertel,Mitte,Berlin,52.519123,13.341872
3,0104,Tiergarten,Mitte,Berlin,50.340922,6.956329
4,0105,Wedding,Mitte,Berlin,52.550123,13.341970
...,...,...,...,...,...,...
91,1207,Waidmannslust,Reinickendorf,Berlin,52.608035,13.322533
92,1208,Lübars,Reinickendorf,Berlin,52.614647,13.353020
93,1209,Wittenau,Reinickendorf,Berlin,52.591237,13.323320
94,1210,Märkisches Viertel,Reinickendorf,Berlin,52.599312,13.356532


Create a map of berlin with neighborhoods details.

In [10]:
# create map of Cologne using latitude and longitude 
map_berlin = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to the map
for lat, lng, label in zip(df_berlin['Latitude'], df_berlin['Longitude'], df_berlin['neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_berlin) 
    
map_berlin

## 3.3 Data analysis using Foursquare API

The aim of this part is getting details using data exploraty analysis in order to extract valuable information and insights about all these 96 different neigborhoods. The aim is getting rich information which could help us to make the rights decisions.

### 3.3.1 First of all we will use Foursquare API to explore the neighborhoods of Berlin and segment them.

In [11]:
CLIENT_ID = 'BR3G0GSMYNJNDMJMI4VBRWOC3JY0ETEQZEH2FQ4QH0XXDLZM' # your Foursquare ID
CLIENT_SECRET = 'MOU5CFO22YMABCZ3XBNZAFWEPLBU5LPVIPX3YXYCCUPFKNG0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BR3G0GSMYNJNDMJMI4VBRWOC3JY0ETEQZEH2FQ4QH0XXDLZM
CLIENT_SECRET:MOU5CFO22YMABCZ3XBNZAFWEPLBU5LPVIPX3YXYCCUPFKNG0


In [12]:
df_berlin.loc[0, 'neighborhood']

'Mitte'

In [13]:
neighborhood_latitude = df_berlin.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_berlin.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_berlin.loc[0, 'neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Mitte are 39.98020495, -7.905590887431517.


In [14]:
# Defining parameters to Foursqaue API

LIMIT = 100
radius = 1000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=BR3G0GSMYNJNDMJMI4VBRWOC3JY0ETEQZEH2FQ4QH0XXDLZM&client_secret=MOU5CFO22YMABCZ3XBNZAFWEPLBU5LPVIPX3YXYCCUPFKNG0&v=20180605&ll=39.98020495,-7.905590887431517&radius=1000&limit=100'

In [15]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60fece794448517028ee32e6'},
 'response': {'warning': {'text': "There aren't a lot of results near you. Try something more general, reset your filters, or expand the search area."},
  'headerLocation': 'Current map view',
  'headerFullLocation': 'Current map view',
  'headerLocationGranularity': 'unknown',
  'totalResults': 1,
  'suggestedBounds': {'ne': {'lat': 39.98920495900001,
    'lng': -7.893867544095733},
   'sw': {'lat': 39.97120494099999, 'lng': -7.917314230767301}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5b9642f665211f002c799be8',
       'name': 'Yoga Evolution Retreats',
       'location': {'address': 'Yoga Evolution Retreats',
        'crossStreet': 'Quinta Do Bacelo',
        'lat': 39.979904,
        'lng': -7.91

In [16]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [17]:
import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [18]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,name,categories,lat,lng
0,Yoga Evolution Retreats,Yoga Studio,39.979904,-7.914809


In [19]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

1 venues were returned by Foursquare.


In [20]:
print ('{} unique categories in Mitte.'.format(nearby_venues['categories'].value_counts().shape[0]))

1 unique categories in Mitte.


In [21]:
print (nearby_venues['categories'].value_counts()[0:15])

Yoga Studio    1
Name: categories, dtype: int64


In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=3000, LIMIT=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']

    
    return(nearby_venues)

In [25]:
berlin_venues = getNearbyVenues(names=df_berlin['neighborhood'],
                                   latitudes=df_berlin['Latitude'],
                                   longitudes=df_berlin['Longitude']
                                  )

Mitte
Moabit
Hansaviertel
Tiergarten
Wedding
Gesundbrunnen
Friedrichshain
Kreuzberg
Prenzlauer Berg
Weißensee
Blankenburg
Heinersdorf
Karow
Stadtrandsiedlung Malchow
Pankow
Blankenfelde
Buch
Französisch Buchholz
Niederschönhausen
Rosenthal
Wilhelmsruh
Charlottenburg
Wilmersdorf
Schmargendorf
Grunewald
Westend
Charlottenburg-Nord
Halensee
Spandau
Haselhorst
Siemensstadt
Staaken
Gatow
Kladow
Hakenfelde
Falkenhagener Feld
Wilhelmstadt
Steglitz
Lichterfelde
Lankwitz
Zehlendorf
Dahlem
Nikolassee
Wannsee
Schöneberg
Friedenau
Tempelhof
Mariendorf
Marienfelde
Lichtenrade
Neukölln
Britz
Buckow
Rudow
Gropiusstadt
Alt-Treptow
Plänterwald
Baumschulenweg
Johannisthal
Niederschöneweide
Altglienicke
Adlershof
Bohnsdorf
Oberschöneweide
Köpenick
Friedrichshagen
Rahnsdorf
Grünau
Müggelheim
Schmöckwitz
Marzahn
Biesdorf
Kaulsdorf
Mahlsdorf
Hellersdorf
Friedrichsfelde
Karlshorst
Lichtenberg
Falkenberg
Malchow
Wartenberg
Neu-Hohenschönhausen
Alt-Hohenschönhausen
Fennpfuhl
Rummelsburg
Reinickendorf
Tegel
Kon

In [26]:
berlin_venues.shape

(6243, 7)

In [27]:
# Build a dataframe to concentrate only on Restaurants
Berlin_Venues_only_restaurants = berlin_venues[berlin_venues['Venue Category'].str.contains('Restaurant')].reset_index(drop=True)
Berlin_Venues_only_restaurants.index = np.arange(1, len(Berlin_Venues_only_restaurants )+1)

In [28]:
print (Berlin_Venues_only_restaurants['Venue Category'].value_counts())

Italian Restaurant                 240
German Restaurant                  174
Greek Restaurant                    86
Vietnamese Restaurant               78
Restaurant                          75
Fast Food Restaurant                57
Falafel Restaurant                  52
Indian Restaurant                   49
Chinese Restaurant                  47
Thai Restaurant                     43
Doner Restaurant                    42
Asian Restaurant                    39
Sushi Restaurant                    38
Seafood Restaurant                  30
Korean Restaurant                   29
Turkish Restaurant                  27
Vegetarian / Vegan Restaurant       26
Argentinian Restaurant              23
Mexican Restaurant                  23
Middle Eastern Restaurant           21
Japanese Restaurant                 20
Eastern European Restaurant         19
French Restaurant                   12
African Restaurant                   9
American Restaurant                  9
Modern European Restauran

In [29]:
Berlin_Venues_only_restaurants.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1,Moabit,52.530102,13.342542,Güllü Lahmacun,52.532217,13.350061,Turkish Restaurant
2,Moabit,52.530102,13.342542,Sapori Di Casa,52.527425,13.351519,Italian Restaurant
3,Moabit,52.530102,13.342542,Favorit Gemüse Kebap,52.526737,13.335998,Doner Restaurant
4,Moabit,52.530102,13.342542,Hoan Kiem,52.527047,13.338970,Vietnamese Restaurant
5,Moabit,52.530102,13.342542,Agni,52.524497,13.327126,Indian Restaurant


### 3.3.2 Analysis of the neighbourhoods.

In [30]:
Berlin_Venues_restaurants = Berlin_Venues_only_restaurants.groupby(['Neighborhood'])['Venue Category'].apply(lambda x: x[x.str.contains('Restaurant')].count())

In [31]:
Berlin_Venues_restaurants

Neighborhood
Adlershof               19
Alt-Hohenschönhausen    11
Alt-Treptow             18
Altglienicke            11
Baumschulenweg          18
                        ..
Wilhelmsruh             27
Wilhelmstadt            17
Wilmersdorf             24
Wittenau                31
Zehlendorf              19
Name: Venue Category, Length: 88, dtype: int64

In [32]:
Berlin_Venues_restaurant_df  = Berlin_Venues_restaurants.to_frame().reset_index()
Berlin_Venues_restaurant_df.columns = ['Neighborhood', 'Number of Restaurant']
Berlin_Venues_restaurant_df.index = np.arange(1, len(Berlin_Venues_restaurant_df)+1)
list_rest_no =Berlin_Venues_restaurant_df['Number of Restaurant'].to_list()
list_dist =Berlin_Venues_restaurant_df['Neighborhood'].to_list()

In [33]:
# one hot encoding
Berlin_onehot = pd.get_dummies(Berlin_Venues_only_restaurants[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Berlin_onehot['Neighborhood'] = Berlin_Venues_only_restaurants['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Berlin_onehot.columns[-1]] + list(Berlin_onehot.columns[:-1])
Berlin_onehot = Berlin_onehot[fixed_columns]

Berlin_onehot.head()

,Neighborhood,African Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Austrian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Caucasian Restaurant,...,Swiss Restaurant,Syrian Restaurant,Szechuan Restaurant,Tapas Restaurant,Thai Restaurant,Turkish Home Cooking Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yemeni Restaurant
1,Moabit,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,Moabit,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Moabit,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Moabit,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
5,Moabit,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 3.3.3 Grouping by neighbourhoods and introduce the mean for each category of restaurants.

In [34]:
Berlin_grouped = Berlin_onehot.groupby('Neighborhood').mean().reset_index()
Berlin_grouped

,Neighborhood,African Restaurant,American Restaurant,Argentinian Restaurant,Asian Restaurant,Austrian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Caucasian Restaurant,...,Swiss Restaurant,Syrian Restaurant,Szechuan Restaurant,Tapas Restaurant,Thai Restaurant,Turkish Home Cooking Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Yemeni Restaurant
0,Adlershof,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
1,Alt-Hohenschönhausen,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.363636,0.0
2,Alt-Treptow,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.055556,0.111111,0.0,0.000000,0.111111,0.055556,0.0
3,Altglienicke,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0
4,Baumschulenweg,0.0,0.000000,0.000000,0.055556,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.055556,0.055556,0.111111,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,Wilhelmsruh,0.0,0.037037,0.037037,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.037037,0.0,0.037037,0.000000,0.037037,0.0
84,Wilhelmstadt,0.0,0.000000,0.058824,0.000000,0.058824,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.117647,0.0
85,Wilmersdorf,0.0,0.000000,0.000000,0.083333,0.000000,0.0,0.0,0.041667,0.000000,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.125000,0.0
86,Wittenau,0.0,0.032258,0.064516,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.000000,0.032258,0.0,0.000000,0.000000,0.000000,0.0


In [35]:
Berlin_grouped.shape

(88, 57)

### 3.3.4 Top 10 most common venues with their respective restaurants

In [36]:
num_top_venues = 10

for hood in Berlin_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Berlin_grouped[Berlin_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adlershof----
                        venue  freq
0           German Restaurant  0.26
1            Greek Restaurant  0.21
2          Italian Restaurant  0.16
3            Sushi Restaurant  0.16
4   Middle Eastern Restaurant  0.05
5           Korean Restaurant  0.05
6           Indian Restaurant  0.05
7                  Restaurant  0.05
8          Russian Restaurant  0.00
9  Modern European Restaurant  0.00


----Alt-Hohenschönhausen----
                        venue  freq
0       Vietnamese Restaurant  0.36
1           German Restaurant  0.27
2           Indian Restaurant  0.18
3          Italian Restaurant  0.09
4            Greek Restaurant  0.09
5          African Restaurant  0.00
6          Russian Restaurant  0.00
7   Middle Eastern Restaurant  0.00
8  Modern European Restaurant  0.00
9     New American Restaurant  0.00


----Alt-Treptow----
                           venue  freq
0             Falafel Restaurant  0.22
1  Vegetarian / Vegan Restaurant  0.11
2                Tha

### 3.3.5 Creating a dataframe.

In [37]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [38]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Berlin_grouped['Neighborhood']

for ind in np.arange(Berlin_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Berlin_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(20)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adlershof,German Restaurant,Greek Restaurant,Italian Restaurant,Sushi Restaurant,Middle Eastern Restaurant,Korean Restaurant,Indian Restaurant,Restaurant,Russian Restaurant,Modern European Restaurant
1,Alt-Hohenschönhausen,Vietnamese Restaurant,German Restaurant,Indian Restaurant,Italian Restaurant,Greek Restaurant,African Restaurant,Russian Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant
2,Alt-Treptow,Falafel Restaurant,Vegetarian / Vegan Restaurant,Thai Restaurant,Italian Restaurant,Lebanese Restaurant,German Restaurant,Dumpling Restaurant,Spanish Restaurant,Russian Restaurant,Tapas Restaurant
3,Altglienicke,Greek Restaurant,Italian Restaurant,Korean Restaurant,Chinese Restaurant,Sushi Restaurant,German Restaurant,Restaurant,African Restaurant,Schnitzel Restaurant,Modern European Restaurant
4,Baumschulenweg,Falafel Restaurant,Vietnamese Restaurant,Italian Restaurant,German Restaurant,Chinese Restaurant,Indian Restaurant,Fast Food Restaurant,Sushi Restaurant,Doner Restaurant,Mexican Restaurant
5,Biesdorf,German Restaurant,African Restaurant,Lebanese Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,Persian Restaurant,Peruvian Restaurant,Portuguese Restaurant
6,Blankenburg,German Restaurant,African Restaurant,Lebanese Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,Persian Restaurant,Peruvian Restaurant,Portuguese Restaurant
7,Blankenfelde,Mexican Restaurant,Greek Restaurant,German Restaurant,Restaurant,African Restaurant,Schnitzel Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,Persian Restaurant
8,Bohnsdorf,Chinese Restaurant,Greek Restaurant,African Restaurant,Seafood Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,Persian Restaurant,Peruvian Restaurant
9,Borsigwalde,German Restaurant,Restaurant,Italian Restaurant,Eastern European Restaurant,Indian Restaurant,Falafel Restaurant,Argentinian Restaurant,Thai Restaurant,Seafood Restaurant,Japanese Restaurant


## 3.4 Clustering the neighbourhoods with k-means.

In [39]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [40]:
# set number of clusters (5)
kclusters = 5

Berlin_grouped_clustering = Berlin_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Berlin_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only


array([0, 0, 0, 3, 0, 1, 1, 3, 3, 0], dtype=int32)

In [41]:
Berlin_merged = df_berlin
Berlin_merged.head(10)

,neighborhood_id,neighborhood,borough,city,Latitude,Longitude
0,0101,Mitte,Mitte,Berlin,39.980205,-7.905591
1,0102,Moabit,Mitte,Berlin,52.530102,13.342542
2,0103,Hansaviertel,Mitte,Berlin,52.519123,13.341872
3,0104,Tiergarten,Mitte,Berlin,50.340922,6.956329
4,0105,Wedding,Mitte,Berlin,52.550123,13.341970
5,0106,Gesundbrunnen,Mitte,Berlin,52.550920,13.384846
6,0201,Friedrichshain,Friedrichshain-Kreuzberg,Berlin,52.512215,13.450290
7,0202,Kreuzberg,Friedrichshain-Kreuzberg,Berlin,52.497644,13.411914
8,0301,Prenzlauer Berg,Pankow,Berlin,52.539847,13.428565
9,0302,Weißensee,Pankow,Berlin,52.554619,13.463002


In [42]:
neighborhoods_venues_sorted.head(20)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adlershof,German Restaurant,Greek Restaurant,Italian Restaurant,Sushi Restaurant,Middle Eastern Restaurant,Korean Restaurant,Indian Restaurant,Restaurant,Russian Restaurant,Modern European Restaurant
1,Alt-Hohenschönhausen,Vietnamese Restaurant,German Restaurant,Indian Restaurant,Italian Restaurant,Greek Restaurant,African Restaurant,Russian Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant
2,Alt-Treptow,Falafel Restaurant,Vegetarian / Vegan Restaurant,Thai Restaurant,Italian Restaurant,Lebanese Restaurant,German Restaurant,Dumpling Restaurant,Spanish Restaurant,Russian Restaurant,Tapas Restaurant
3,Altglienicke,Greek Restaurant,Italian Restaurant,Korean Restaurant,Chinese Restaurant,Sushi Restaurant,German Restaurant,Restaurant,African Restaurant,Schnitzel Restaurant,Modern European Restaurant
4,Baumschulenweg,Falafel Restaurant,Vietnamese Restaurant,Italian Restaurant,German Restaurant,Chinese Restaurant,Indian Restaurant,Fast Food Restaurant,Sushi Restaurant,Doner Restaurant,Mexican Restaurant
5,Biesdorf,German Restaurant,African Restaurant,Lebanese Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,Persian Restaurant,Peruvian Restaurant,Portuguese Restaurant
6,Blankenburg,German Restaurant,African Restaurant,Lebanese Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,Persian Restaurant,Peruvian Restaurant,Portuguese Restaurant
7,Blankenfelde,Mexican Restaurant,Greek Restaurant,German Restaurant,Restaurant,African Restaurant,Schnitzel Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,Persian Restaurant
8,Bohnsdorf,Chinese Restaurant,Greek Restaurant,African Restaurant,Seafood Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,Persian Restaurant,Peruvian Restaurant
9,Borsigwalde,German Restaurant,Restaurant,Italian Restaurant,Eastern European Restaurant,Indian Restaurant,Falafel Restaurant,Argentinian Restaurant,Thai Restaurant,Seafood Restaurant,Japanese Restaurant


In [43]:
neighborhoods_venues_sorted_w_clusters = neighborhoods_venues_sorted

In [44]:
neighborhoods_venues_sorted_w_clusters.head(9)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adlershof,German Restaurant,Greek Restaurant,Italian Restaurant,Sushi Restaurant,Middle Eastern Restaurant,Korean Restaurant,Indian Restaurant,Restaurant,Russian Restaurant,Modern European Restaurant
1,Alt-Hohenschönhausen,Vietnamese Restaurant,German Restaurant,Indian Restaurant,Italian Restaurant,Greek Restaurant,African Restaurant,Russian Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant
2,Alt-Treptow,Falafel Restaurant,Vegetarian / Vegan Restaurant,Thai Restaurant,Italian Restaurant,Lebanese Restaurant,German Restaurant,Dumpling Restaurant,Spanish Restaurant,Russian Restaurant,Tapas Restaurant
3,Altglienicke,Greek Restaurant,Italian Restaurant,Korean Restaurant,Chinese Restaurant,Sushi Restaurant,German Restaurant,Restaurant,African Restaurant,Schnitzel Restaurant,Modern European Restaurant
4,Baumschulenweg,Falafel Restaurant,Vietnamese Restaurant,Italian Restaurant,German Restaurant,Chinese Restaurant,Indian Restaurant,Fast Food Restaurant,Sushi Restaurant,Doner Restaurant,Mexican Restaurant
5,Biesdorf,German Restaurant,African Restaurant,Lebanese Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,Persian Restaurant,Peruvian Restaurant,Portuguese Restaurant
6,Blankenburg,German Restaurant,African Restaurant,Lebanese Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,Persian Restaurant,Peruvian Restaurant,Portuguese Restaurant
7,Blankenfelde,Mexican Restaurant,Greek Restaurant,German Restaurant,Restaurant,African Restaurant,Schnitzel Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,Persian Restaurant
8,Bohnsdorf,Chinese Restaurant,Greek Restaurant,African Restaurant,Seafood Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,Persian Restaurant,Peruvian Restaurant


In [45]:
# add clustering labels
neighborhoods_venues_sorted_w_clusters.insert(0, 'Cluster Labels', kmeans.labels_)

In [46]:
neighborhoods_venues_sorted_w_clusters.head(23)

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Adlershof,German Restaurant,Greek Restaurant,Italian Restaurant,Sushi Restaurant,Middle Eastern Restaurant,Korean Restaurant,Indian Restaurant,Restaurant,Russian Restaurant,Modern European Restaurant
1,0,Alt-Hohenschönhausen,Vietnamese Restaurant,German Restaurant,Indian Restaurant,Italian Restaurant,Greek Restaurant,African Restaurant,Russian Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant
2,0,Alt-Treptow,Falafel Restaurant,Vegetarian / Vegan Restaurant,Thai Restaurant,Italian Restaurant,Lebanese Restaurant,German Restaurant,Dumpling Restaurant,Spanish Restaurant,Russian Restaurant,Tapas Restaurant
3,3,Altglienicke,Greek Restaurant,Italian Restaurant,Korean Restaurant,Chinese Restaurant,Sushi Restaurant,German Restaurant,Restaurant,African Restaurant,Schnitzel Restaurant,Modern European Restaurant
4,0,Baumschulenweg,Falafel Restaurant,Vietnamese Restaurant,Italian Restaurant,German Restaurant,Chinese Restaurant,Indian Restaurant,Fast Food Restaurant,Sushi Restaurant,Doner Restaurant,Mexican Restaurant
5,1,Biesdorf,German Restaurant,African Restaurant,Lebanese Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,Persian Restaurant,Peruvian Restaurant,Portuguese Restaurant
6,1,Blankenburg,German Restaurant,African Restaurant,Lebanese Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,Persian Restaurant,Peruvian Restaurant,Portuguese Restaurant
7,3,Blankenfelde,Mexican Restaurant,Greek Restaurant,German Restaurant,Restaurant,African Restaurant,Schnitzel Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,Persian Restaurant
8,3,Bohnsdorf,Chinese Restaurant,Greek Restaurant,African Restaurant,Seafood Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,Persian Restaurant,Peruvian Restaurant
9,0,Borsigwalde,German Restaurant,Restaurant,Italian Restaurant,Eastern European Restaurant,Indian Restaurant,Falafel Restaurant,Argentinian Restaurant,Thai Restaurant,Seafood Restaurant,Japanese Restaurant


In [47]:
Berlin_merged.rename(columns={'neighborhood':'Neighborhood'}, inplace=True)

Berlin_merged = Berlin_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Berlin_merged.head(10)

,neighborhood_id,Neighborhood,borough,city,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0101,Mitte,Mitte,Berlin,39.980205,-7.905591,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0102,Moabit,Mitte,Berlin,52.530102,13.342542,0.0,Vietnamese Restaurant,Turkish Restaurant,Chinese Restaurant,African Restaurant,Doner Restaurant,Persian Restaurant,Kebab Restaurant,Italian Restaurant,Indian Restaurant,Schnitzel Restaurant
2,0103,Hansaviertel,Mitte,Berlin,52.519123,13.341872,0.0,Seafood Restaurant,Doner Restaurant,Kebab Restaurant,Italian Restaurant,Indian Restaurant,Restaurant,German Restaurant,French Restaurant,Schnitzel Restaurant,Middle Eastern Restaurant
3,0104,Tiergarten,Mitte,Berlin,50.340922,6.956329,2.0,German Restaurant,Eastern European Restaurant,Restaurant,Seafood Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,Persian Restaurant,Peruvian Restaurant
4,0105,Wedding,Mitte,Berlin,52.550123,13.341970,0.0,Italian Restaurant,Turkish Restaurant,Restaurant,Chinese Restaurant,Thai Restaurant,Middle Eastern Restaurant,African Restaurant,Falafel Restaurant,Vietnamese Restaurant,Modern European Restaurant
5,0106,Gesundbrunnen,Mitte,Berlin,52.550920,13.384846,0.0,Seafood Restaurant,Vietnamese Restaurant,Sushi Restaurant,Modern European Restaurant,Italian Restaurant,French Restaurant,Falafel Restaurant,Doner Restaurant,African Restaurant,Tapas Restaurant
6,0201,Friedrichshain,Friedrichshain-Kreuzberg,Berlin,52.512215,13.450290,0.0,Italian Restaurant,Falafel Restaurant,Vegetarian / Vegan Restaurant,Thai Restaurant,Middle Eastern Restaurant,Mexican Restaurant,Russian Restaurant,Syrian Restaurant,Chinese Restaurant,Modern European Restaurant
7,0202,Kreuzberg,Friedrichshain-Kreuzberg,Berlin,52.497644,13.411914,0.0,Turkish Restaurant,Italian Restaurant,Vietnamese Restaurant,Spanish Restaurant,German Restaurant,African Restaurant,Mediterranean Restaurant,Lebanese Restaurant,Kumpir Restaurant,Korean Restaurant
8,0301,Prenzlauer Berg,Pankow,Berlin,52.539847,13.428565,0.0,Vietnamese Restaurant,Falafel Restaurant,Japanese Restaurant,Indian Restaurant,Italian Restaurant,Empanada Restaurant,Sushi Restaurant,Dumpling Restaurant,Doner Restaurant,Seafood Restaurant
9,0302,Weißensee,Pankow,Berlin,52.554619,13.463002,0.0,German Restaurant,Indian Restaurant,Greek Restaurant,Restaurant,Fast Food Restaurant,Dim Sum Restaurant,Italian Restaurant,Falafel Restaurant,Vegetarian / Vegan Restaurant,Ramen Restaurant


Finally, let's visualize the resulting clusters.

In [48]:
# Delete those Neighborhood not classified = NaN in Cluster Labels

Berlin_merged = Berlin_merged[Berlin_merged['Cluster Labels'].notna()]
Berlin_merged.head(10)

,neighborhood_id,Neighborhood,borough,city,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,0102,Moabit,Mitte,Berlin,52.530102,13.342542,0.0,Vietnamese Restaurant,Turkish Restaurant,Chinese Restaurant,African Restaurant,Doner Restaurant,Persian Restaurant,Kebab Restaurant,Italian Restaurant,Indian Restaurant,Schnitzel Restaurant
2,0103,Hansaviertel,Mitte,Berlin,52.519123,13.341872,0.0,Seafood Restaurant,Doner Restaurant,Kebab Restaurant,Italian Restaurant,Indian Restaurant,Restaurant,German Restaurant,French Restaurant,Schnitzel Restaurant,Middle Eastern Restaurant
3,0104,Tiergarten,Mitte,Berlin,50.340922,6.956329,2.0,German Restaurant,Eastern European Restaurant,Restaurant,Seafood Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,Persian Restaurant,Peruvian Restaurant
4,0105,Wedding,Mitte,Berlin,52.550123,13.341970,0.0,Italian Restaurant,Turkish Restaurant,Restaurant,Chinese Restaurant,Thai Restaurant,Middle Eastern Restaurant,African Restaurant,Falafel Restaurant,Vietnamese Restaurant,Modern European Restaurant
5,0106,Gesundbrunnen,Mitte,Berlin,52.550920,13.384846,0.0,Seafood Restaurant,Vietnamese Restaurant,Sushi Restaurant,Modern European Restaurant,Italian Restaurant,French Restaurant,Falafel Restaurant,Doner Restaurant,African Restaurant,Tapas Restaurant
6,0201,Friedrichshain,Friedrichshain-Kreuzberg,Berlin,52.512215,13.450290,0.0,Italian Restaurant,Falafel Restaurant,Vegetarian / Vegan Restaurant,Thai Restaurant,Middle Eastern Restaurant,Mexican Restaurant,Russian Restaurant,Syrian Restaurant,Chinese Restaurant,Modern European Restaurant
7,0202,Kreuzberg,Friedrichshain-Kreuzberg,Berlin,52.497644,13.411914,0.0,Turkish Restaurant,Italian Restaurant,Vietnamese Restaurant,Spanish Restaurant,German Restaurant,African Restaurant,Mediterranean Restaurant,Lebanese Restaurant,Kumpir Restaurant,Korean Restaurant
8,0301,Prenzlauer Berg,Pankow,Berlin,52.539847,13.428565,0.0,Vietnamese Restaurant,Falafel Restaurant,Japanese Restaurant,Indian Restaurant,Italian Restaurant,Empanada Restaurant,Sushi Restaurant,Dumpling Restaurant,Doner Restaurant,Seafood Restaurant
9,0302,Weißensee,Pankow,Berlin,52.554619,13.463002,0.0,German Restaurant,Indian Restaurant,Greek Restaurant,Restaurant,Fast Food Restaurant,Dim Sum Restaurant,Italian Restaurant,Falafel Restaurant,Vegetarian / Vegan Restaurant,Ramen Restaurant
10,0303,Blankenburg,Pankow,Berlin,51.790268,10.955199,1.0,German Restaurant,African Restaurant,Lebanese Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,Persian Restaurant,Peruvian Restaurant,Portuguese Restaurant


In [49]:
# Convert into integer Cluster Labels column

Berlin_merged['Cluster Labels'] = Berlin_merged['Cluster Labels'].astype(int)

In [50]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Berlin_merged['Latitude'], Berlin_merged['Longitude'], Berlin_merged['Neighborhood'], Berlin_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 3.4.1 Exploring clusters

Now, we can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we can then assign a name to each cluster.

**CLUSTER 1**

In [51]:
Berlin_merged.loc[Berlin_merged['Cluster Labels'] == 0, Berlin_merged.columns[[1] + list(range(5, Berlin_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Moabit,13.342542,0,Vietnamese Restaurant,Turkish Restaurant,Chinese Restaurant,African Restaurant,Doner Restaurant,Persian Restaurant,Kebab Restaurant,Italian Restaurant,Indian Restaurant,Schnitzel Restaurant
2,Hansaviertel,13.341872,0,Seafood Restaurant,Doner Restaurant,Kebab Restaurant,Italian Restaurant,Indian Restaurant,Restaurant,German Restaurant,French Restaurant,Schnitzel Restaurant,Middle Eastern Restaurant
4,Wedding,13.341970,0,Italian Restaurant,Turkish Restaurant,Restaurant,Chinese Restaurant,Thai Restaurant,Middle Eastern Restaurant,African Restaurant,Falafel Restaurant,Vietnamese Restaurant,Modern European Restaurant
5,Gesundbrunnen,13.384846,0,Seafood Restaurant,Vietnamese Restaurant,Sushi Restaurant,Modern European Restaurant,Italian Restaurant,French Restaurant,Falafel Restaurant,Doner Restaurant,African Restaurant,Tapas Restaurant
6,Friedrichshain,13.450290,0,Italian Restaurant,Falafel Restaurant,Vegetarian / Vegan Restaurant,Thai Restaurant,Middle Eastern Restaurant,Mexican Restaurant,Russian Restaurant,Syrian Restaurant,Chinese Restaurant,Modern European Restaurant
7,Kreuzberg,13.411914,0,Turkish Restaurant,Italian Restaurant,Vietnamese Restaurant,Spanish Restaurant,German Restaurant,African Restaurant,Mediterranean Restaurant,Lebanese Restaurant,Kumpir Restaurant,Korean Restaurant
8,Prenzlauer Berg,13.428565,0,Vietnamese Restaurant,Falafel Restaurant,Japanese Restaurant,Indian Restaurant,Italian Restaurant,Empanada Restaurant,Sushi Restaurant,Dumpling Restaurant,Doner Restaurant,Seafood Restaurant
9,Weißensee,13.463002,0,German Restaurant,Indian Restaurant,Greek Restaurant,Restaurant,Fast Food Restaurant,Dim Sum Restaurant,Italian Restaurant,Falafel Restaurant,Vegetarian / Vegan Restaurant,Ramen Restaurant
11,Heinersdorf,13.437015,0,Vietnamese Restaurant,Italian Restaurant,Falafel Restaurant,German Restaurant,Indian Restaurant,Fast Food Restaurant,Japanese Restaurant,Dim Sum Restaurant,Korean Restaurant,Chinese Restaurant
21,Charlottenburg,13.309683,0,Italian Restaurant,German Restaurant,Korean Restaurant,Vietnamese Restaurant,Asian Restaurant,Thai Restaurant,Japanese Restaurant,Sushi Restaurant,Greek Restaurant,French Restaurant


**CLUSTER 2**

In [52]:
Berlin_merged.loc[Berlin_merged['Cluster Labels'] == 1, Berlin_merged.columns[[1] + list(range(5, Berlin_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,Blankenburg,10.955199,1,German Restaurant,African Restaurant,Lebanese Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,Persian Restaurant,Peruvian Restaurant,Portuguese Restaurant
71,Biesdorf,6.305603,1,German Restaurant,African Restaurant,Lebanese Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,Persian Restaurant,Peruvian Restaurant,Portuguese Restaurant


**CLUSTER 3**

In [53]:
Berlin_merged.loc[Berlin_merged['Cluster Labels'] == 2, Berlin_merged.columns[[1] + list(range(5, Berlin_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Tiergarten,6.956329,2,German Restaurant,Eastern European Restaurant,Restaurant,Seafood Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,Persian Restaurant,Peruvian Restaurant
32,Gatow,13.180134,2,German Restaurant,Seafood Restaurant,Italian Restaurant,Schnitzel Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,Persian Restaurant,Peruvian Restaurant
33,Kladow,13.140052,2,German Restaurant,Italian Restaurant,Greek Restaurant,African Restaurant,Schnitzel Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,Persian Restaurant,Peruvian Restaurant
42,Nikolassee,13.198145,2,German Restaurant,Italian Restaurant,Restaurant,Asian Restaurant,Indian Restaurant,Fast Food Restaurant,African Restaurant,Schnitzel Restaurant,Modern European Restaurant,New American Restaurant
43,Wannsee,13.158937,2,German Restaurant,Italian Restaurant,Asian Restaurant,Austrian Restaurant,Indian Restaurant,Fast Food Restaurant,Chinese Restaurant,Restaurant,African Restaurant,Schnitzel Restaurant
64,Köpenick,13.576413,2,German Restaurant,Italian Restaurant,Greek Restaurant,Middle Eastern Restaurant,Indian Restaurant,Sushi Restaurant,African Restaurant,Russian Restaurant,Modern European Restaurant,New American Restaurant
66,Rahnsdorf,13.701802,2,German Restaurant,Seafood Restaurant,Fast Food Restaurant,Schnitzel Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,Persian Restaurant,Peruvian Restaurant
68,Müggelheim,13.661954,2,American Restaurant,German Restaurant,Seafood Restaurant,Vegetarian / Vegan Restaurant,Turkish Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant
81,Neu-Hohenschönhausen,13.514065,2,German Restaurant,Indian Restaurant,Greek Restaurant,African Restaurant,Schnitzel Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,Persian Restaurant,Peruvian Restaurant
87,Konradshöhe,13.223198,2,Turkish Restaurant,German Restaurant,Restaurant,African Restaurant,Schnitzel Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,Persian Restaurant


**CLUSTER 4**

In [54]:
Berlin_merged.loc[Berlin_merged['Cluster Labels'] == 3, Berlin_merged.columns[[1] + list(range(5, Berlin_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Stadtrandsiedlung Malchow,13.463285,3,Greek Restaurant,Restaurant,Fast Food Restaurant,African Restaurant,Schnitzel Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,Persian Restaurant
15,Blankenfelde,13.388447,3,Mexican Restaurant,Greek Restaurant,German Restaurant,Restaurant,African Restaurant,Schnitzel Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,Persian Restaurant
17,Französisch Buchholz,13.428110,3,Mexican Restaurant,Asian Restaurant,Greek Restaurant,Doner Restaurant,Seafood Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,Persian Restaurant,Peruvian Restaurant
60,Altglienicke,13.542646,3,Greek Restaurant,Italian Restaurant,Korean Restaurant,Chinese Restaurant,Sushi Restaurant,German Restaurant,Restaurant,African Restaurant,Schnitzel Restaurant,Modern European Restaurant
62,Bohnsdorf,13.570665,3,Chinese Restaurant,Greek Restaurant,African Restaurant,Seafood Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,Persian Restaurant,Peruvian Restaurant
73,Mahlsdorf,13.613162,3,Greek Restaurant,Fast Food Restaurant,Italian Restaurant,Chinese Restaurant,African Restaurant,Schnitzel Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,Persian Restaurant
74,Hellersdorf,13.604774,3,Greek Restaurant,Fast Food Restaurant,Falafel Restaurant,Asian Restaurant,Italian Restaurant,Mexican Restaurant,African Restaurant,Modern European Restaurant,New American Restaurant,Persian Restaurant
80,Wartenberg,13.517582,3,Greek Restaurant,Fast Food Restaurant,African Restaurant,Schnitzel Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,Persian Restaurant,Peruvian Restaurant


**CLUSTER 5**

In [55]:
Berlin_merged.loc[Berlin_merged['Cluster Labels'] == 4, Berlin_merged.columns[[1] + list(range(5, Berlin_merged.shape[1]))]]

,Neighborhood,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,Karow,13.486276,4,Italian Restaurant,Fast Food Restaurant,African Restaurant,Schnitzel Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant,Persian Restaurant,Peruvian Restaurant
14,Pankow,13.435316,4,Italian Restaurant,Mexican Restaurant,Asian Restaurant,Thai Restaurant,Doner Restaurant,Greek Restaurant,Restaurant,Schnitzel Restaurant,Middle Eastern Restaurant,Modern European Restaurant
18,Niederschönhausen,13.401397,4,Mexican Restaurant,Italian Restaurant,Asian Restaurant,Modern European Restaurant,Greek Restaurant,German Restaurant,Schnitzel Restaurant,Middle Eastern Restaurant,New American Restaurant,Persian Restaurant
20,Wilhelmsruh,13.362206,4,Italian Restaurant,German Restaurant,Greek Restaurant,Restaurant,Mexican Restaurant,Doner Restaurant,Indian Restaurant,Eastern European Restaurant,American Restaurant,Modern European Restaurant
24,Grunewald,13.263754,4,Italian Restaurant,German Restaurant,Chinese Restaurant,Vietnamese Restaurant,Restaurant,Swiss Restaurant,Indian Restaurant,Eastern European Restaurant,Japanese Restaurant,Doner Restaurant
25,Westend,13.255842,4,Italian Restaurant,German Restaurant,Vietnamese Restaurant,Doner Restaurant,Falafel Restaurant,Halal Restaurant,Indian Restaurant,Chinese Restaurant,Austrian Restaurant,Argentinian Restaurant
29,Haselhorst,13.231156,4,German Restaurant,Italian Restaurant,Fast Food Restaurant,Argentinian Restaurant,Asian Restaurant,Vietnamese Restaurant,Greek Restaurant,Turkish Restaurant,Thai Restaurant,Restaurant
31,Staaken,13.143367,4,Fast Food Restaurant,Turkish Restaurant,Italian Restaurant,Chinese Restaurant,Restaurant,African Restaurant,Schnitzel Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant
35,Falkenhagener Feld,13.166894,4,Turkish Restaurant,Italian Restaurant,Greek Restaurant,Restaurant,Fast Food Restaurant,African Restaurant,Schnitzel Restaurant,Middle Eastern Restaurant,Modern European Restaurant,New American Restaurant
36,Wilhelmstadt,13.191452,4,Italian Restaurant,German Restaurant,Restaurant,Vietnamese Restaurant,Seafood Restaurant,Doner Restaurant,Chinese Restaurant,Austrian Restaurant,Argentinian Restaurant,Persian Restaurant
